In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Correct architecture based on pre-trained MobileNetV2
class MobileNetStudent(nn.Module):
    def __init__(self, num_classes=2):
        super(MobileNetStudent, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True)  # <- Use pretrained=True if model was trained that way
        self.model.classifier[1] = nn.Linear(self.model.last_channel, num_classes)

    def forward(self, x):
        return self.model(x)

# Load the model
model = MobileNetStudent(num_classes=2)

# Load weights
state_dict = torch.load("mobilenet_student_distilled_1.pth", map_location='cpu')
model.load_state_dict(state_dict, strict=False)  # Use strict=False to skip mismatched keys
model.eval()

# Define transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load and preprocess image
img_path = "dfsjh.jpeg"  # Change this to your actual image path
image = Image.open(img_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Predict
with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted = torch.max(outputs, 1)

print("Predicted class index:", predicted.item())


c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\tflite-env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\tflite-env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted class index: 0


In [9]:
import torch
from torchvision import transforms
from PIL import Image
from pathlib import Path
import pandas as pd

# Define your model class or import it if defined elsewhere
# For example, if you used MobileNet, define or import it here
from torchvision.models import mobilenet_v2

# Set device (cpu or gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model architecture
model = mobilenet_v2(pretrained=False)  # or your custom model
num_classes = 2  # adjust to your number of classes

# Modify last layer for your classes if needed
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)

# Load saved weights
model_path = Path("mobilenet_student_distilled_1.pth")
if model_path.exists():
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print("Model loaded successfully")
else:
    raise FileNotFoundError(f"Model not found at {model_path}")

# Define preprocessing transforms similar to your training
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # normalize with mean and std used during training
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Prediction function
def get_predictions(image_path, threshold=0.85):
    """
    Returns:
        flag (int): 1 if class 1 probability >= threshold else 0
        recaptured_probability (float): probability of class 1
        pred_class (str): predicted class label
    """
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0).to(device)  # add batch dim

    # Forward pass
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.softmax(outputs, dim=1)
    
    recaptured_probability = probs[0, 1].item()
    pred_idx = torch.argmax(probs, dim=1).item()
    pred_class = str(pred_idx)  # Or map index to label name here

    flag = 1 if recaptured_probability >= threshold else 0

    return flag, recaptured_probability, pred_class

# Example usage:
# flag, prob, cls = get_predictions('test_image.jpg')
# print(flag, prob, cls)


Model loaded successfully


In [10]:
# Define base folder with images
base_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder")

# Get all images (limit to first 5000)
image_paths = list(base_path.glob('*.jpeg')) + list(base_path.glob('*.jpg')) + list(base_path.glob('*.png'))
# image_paths = image_paths[:5]

print(f"Found {len(image_paths)} images. Starting prediction..\n")

# Prediction loop
results = []

for img_path in image_paths:
    try:
        flag, probability, pred_class = get_predictions(img_path)
        results.append({
            'Image': str(img_path),
            'Class': pred_class,
            'Flag': flag,
            'Recaptured_Probability': probability
        })
    except Exception as e:
        print(f"Error with image {img_path.name}: {e}")

# Create DataFrame and save to CSV
df_results = pd.DataFrame(results)
print("\n Sample results:")
print(df_results.head())

Found 17 images. Starting prediction..


 Sample results:
                                               Image Class  Flag  \
0  C:\Users\arpitasharma2\OneDrive - Deloitte (O3...     0     0   
1  C:\Users\arpitasharma2\OneDrive - Deloitte (O3...     0     0   
2  C:\Users\arpitasharma2\OneDrive - Deloitte (O3...     1     1   
3  C:\Users\arpitasharma2\OneDrive - Deloitte (O3...     0     0   
4  C:\Users\arpitasharma2\OneDrive - Deloitte (O3...     0     0   

   Recaptured_Probability  
0                0.000018  
1                0.000135  
2                0.999669  
3                0.000782  
4                0.003762  


In [11]:
df_results.head()

,Image,Class,Flag,Recaptured_Probability
0,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,0,0,0.000018
1,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,0,0,0.000135
2,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,1,1,0.999669
3,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,0,0,0.000782
4,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,0,0,0.003762


In [14]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=